In [1]:
!pip install torch

In [2]:
import torch.nn.functional as F
import torch.optim as optim
import torch 
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import random



In [3]:
#Evaluate the model in GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
torch.cuda.set_device(device)

cuda:0


In [19]:
#load the mnist data
train_set = torchvision.datasets.FashionMNIST(
    root='./data',
    train=True,
    download=True,
    transform = transforms.Compose([
          transforms.ToTensor()
    ])
)


In [20]:
#Random numberf generation
def getRandomNumber(batchsize):
  return random.sample(range(0, 10), batchsize)

In [6]:
#compare the actual label with predicted label
def get_num_correct(preds, labels):
  return preds.argmax(dim=1).eq(labels).sum().item()

In [21]:
#Model
class Network(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5) 
    self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
    self.fc1 = nn.Linear(in_features=232, out_features=120)
    self.fc2 = nn.Linear(in_features=120, out_features=60)
    self.out = nn.Linear(in_features=60, out_features=29)

    self.fcr1 = nn.Linear(in_features=10, out_features=20)
    self.fcr2 = nn.Linear(in_features=20, out_features=40)
  
  def forward(self, t, randomNum):
    # input layer
    x = t
    y=randomNum


    # conv1 layer
    x = self.conv1(x)
    x = F.relu(x)
    x = F.max_pool2d(x, kernel_size=2, stride=2) # 28 | 24 | 12

    # conv2 layer
    x = self.conv2(x)
    x = F.relu(x)
    x = F.max_pool2d(x, kernel_size=2, stride=2) # 12 | 8 | 4 >> 12x4x4

    # reshapre
    x = x.reshape(-1, 12 * 4 * 4)  # 1*192
    y = y.reshape(-1, 10)

    #print(x.shape)
    #print(y.shape)

    # fcr layer
    y = self.fcr1(y) #1*20
    y = F.relu(y) #1*20


    # fcr layer
    y = self.fcr2(y) #1*40
    y = F.relu(y) #1*40



    x=torch.cat((x,y ), 1)  #1*(40+192) -> 1*232



    # fc1 layer
    x = self.fc1(x) #1*120
    x = F.relu(x)  #1*120

    # fc2 layer
    x = self.fc2(x) #1*60
    x = F.relu(x)   #1*60




    # output layer
    x = self.out(x) #1*29
    return x

In [22]:
network = Network()


In [9]:
from torchsummary import summary

In [10]:
summary(network, [(1,28,28),(1,1,10)])


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 24, 24]             156
            Conv2d-2             [-1, 12, 8, 8]           1,812
            Linear-3                   [-1, 20]             220
            Linear-4                   [-1, 40]             840
            Linear-5                  [-1, 120]          27,960
            Linear-6                   [-1, 60]           7,260
            Linear-7                   [-1, 29]           1,769
Total params: 40,017
Trainable params: 40,017
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.03
Forward/backward pass size (MB): 0.03
Params size (MB): 0.15
Estimated Total Size (MB): 0.22
----------------------------------------------------------------


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [ ]:
#model training 
batch_size=10
train_loader = torch.utils.data.DataLoader(train_set, batch_size) #load the traing data with specified barch size
optimizer = optim.Adam(network.parameters(), lr=0.01) #adam optimizer with the learning rate .01
sigmoid = nn.Sigmoid() # initialize sigmoid layer
lossCal = nn.BCEWithLogitsLoss() # initialize loss function

for epoch in range(50):

    total_loss = 0
    total_correct = 0

    for batch in train_loader: # Get Batch
        images, labels = batch 
        #print("One hot ")


        randomNumTensor = torch.as_tensor(getRandomNumber(batch_size))   
        actualabels=torch.cat((F.one_hot(labels, num_classes=10), F.one_hot(randomNumTensor + labels, num_classes=19)), 1).type(torch.float)  
        print(randomNumTensor)
        print(actualabels)
        print(labels)
        print(torch.as_tensor(F.one_hot(randomNumTensor, num_classes=10)))



        
        predsLabels = network(images,torch.as_tensor(F.one_hot(randomNumTensor, num_classes=10) , dtype=torch.float)) # Pass Batch
                
        #loss = F.cross_entropy(preds, labels) # Calculate Loss
        loss = lossCal(predsLabels, actualabels) # Calculate Loss

        
      
        print(predsLabels)
        print(predsLabels[:,0:10])
        print(predsLabels[:,10:])
        print("ARGMAX Index")
        print(predsLabels[:,0:10].argmax(dim=1))
        print(predsLabels[:,10:].argmax(dim=1))

        print("Actual Label")
        print(labels)
        print(randomNumTensor+labels)
        optimizer.zero_grad()
        loss.backward() # Calculate Gradients
        optimizer.step() # Update Weights

        total_loss += loss.item()
        total_correct += get_num_correct(predsLabels, labels)
    

    print(
        "epoch", epoch, 
        "total_correct:", total_correct, 
        "loss:", total_loss
    )

Streaming output truncated to the last 5000 lines.
         -123.3490, -136.1908,  -17.9363,  -13.4440,  -23.5435,  -21.0761,
          -36.2209,   -4.4034,  -13.9826,    1.8716,  -17.2723,   -0.9415,
          -49.0571],
        [-140.9227, -183.2516, -245.9246, -178.4279, -163.1585, -158.4156,
         -145.1233, -125.8549, -128.9086,  -61.8953,  -35.3440,  -59.5421,
          -34.8167,  -14.4055,    9.9795,  -36.2102,   -6.3502,  -10.2471,
          -13.4674],
        [ -32.6725,  -34.7911,  -26.3807,   -9.0228,   -7.0936,   -2.5021,
           -1.8635,   -1.6708,   -1.4957,   -1.5140,   -1.7170,   -1.9959,
           -3.1926,   -3.7295,   -6.3371,   -7.4466,  -26.0775,  -25.3886,
          -93.8092],
        [ -96.7988,  -90.3746,  -25.7891,  -45.6178,  -28.5486,  -25.5714,
           -8.0750,   -6.5362,  -31.0176,  -19.1169,    7.7851,  -16.4129,
          -94.4037, -121.8189, -136.5973, -181.0233, -244.4948, -234.0594,
         -216.0932],
        [  -8.9241,   -5.7554,   -5.2893